In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [4]:
class Neuron:
  
  def __init__(self, nin):
    self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
    self.b = Value(random.uniform(-1,1))
  
  def __call__(self, x):
    # w * x + b
    act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
    out = act.tanh()
    return out
  
  def parameters(self):
    return self.w + [self.b]

class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]

In [21]:
# Example 1
x = [2.0, 3.0, -1.0] # inputs
n = MLP(3, [4, 4, 1]) # MLP(inputs, layers=(4 node layer, 4 node layer, 1 node layer - output))
n(x) # passing the inputs to the Multi Layer Perceptron

Value(data=0.8623791608500958)

In [22]:
# Example 2
xs = [
  [2.0, 3.0, -1.0], # expected result = 1.0
  [3.0, -1.0, 0.5], # expected result = -1.0
  [0.5, 1.0, 1.0], # expected result = -1.0
  [1.0, 1.0, -1.0], # expected result = 1.0
] # list of groups of inputs
ys = [1.0, -1.0, -1.0, 1.0] # desired targets for each group of inputs

ypred = [n(x) for x in xs] # predicted results from MLP for each group of inputs
ypred

[Value(data=0.8623791608500958),
 Value(data=0.7935382389662231),
 Value(data=0.3850580852174403),
 Value(data=0.5220837183530346)]

In [23]:
# Calculating Loss value for example 2 - using square function for loss
list_of_losses = [(yout - ygt)**2 for ygt, yout in zip(ys, ypred)]
# zip ex: a = [1,2,3] b = [a,b,c] zip(a,b) = [[1,a],[2,b],[3,c]]
# ygt means Y ground truth which is the expected result

list_of_losses
# returned a list of losses for each group of inputs

loss = sum(list_of_losses)
# total loss = the sum of all losses of expected results

loss

Value(data=5.382508781691849)

In [24]:
# calculating grade of nodes with derivative 

# because ypred is a list of output nodes of the MLP and we apply - and ** to those nodes which are instances of Value that has a -,** method
# it means that the result of list of losses and loss itself are Value objects

# losses = loss of ys0 + loss of ys1 + loss of ys2 + loss of ys3
# loss of ys0 = object MLP + loss function -> loss function = (ys0 - ygt0)**2 -> object MLP is a graph with the output node ys0

# doing backward() on loss will spread the gradient (derivate) to the loss function but also to the MLP
# due to that we have, 1) how much the nodes affect the loss function, but also, 2) how much the nodes affect the output node ys0

loss.backward() # always do one calculation per MLP creation because derivates add up and can acumulate
n.layers[0].neurons[0].w[0].grad
# first layer, first neuron, first weight (one weight per input on a neuron, each neuron has 3 inputs), gradient derivative

0.15226990709761384